In [1]:
var moment = require('moment')
var { constants } = require('../utils')
var { advanceTimeAndBlock } = require('../utils/evm')
var { evm } = require('../utils');
var { toUnit } = require('../utils/bn')
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm')
var { next, clear, bnToNumber } = require('../utils/jupyter')
var { Notebook } = require('../utils/notebook')
var { Keep3r } = require('../utils/common')
var { ethers } = require('hardhat')

oneHundred = toUnit(100)

var $ = new Notebook();
var keep3r = new Keep3r();
//var snapshots = new evm.SnapshotManager()

clear();

In [2]:
next(async () => {
    await $.setup(constants.FORK_BLOCK_NUMBER);
    await keep3r.setup();
    
    job = await(await ethers.getContractFactory('JobForTest')).deploy(keep3r.v2.address)
    provider = await $.newSigner();
    weth = await $.fetch('ERC20ForTest', constants.WETH_ADDRESS)
    uniV3Pool = await $.fetch('IUniswapV3Pool', constants.UNISWAP_V3_ORACLE_POOL )
});

Error: Transaction reverted: trying to deploy a contract whose code is too large
    at Keep3r.constructor (solidity/contracts/Keep3r.sol:12)
    at EthModule._estimateGasAction (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/src/internal/hardhat-network/provider/modules/eth.ts:421:9)
    at HardhatNetworkProvider.request (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/src/internal/hardhat-network/provider/provider.ts:105:18)
    at EthersProviderWrapper.send (/Users/levihernan/wonder/keep3r/notebooks/node_modules/@nomiclabs/hardhat-ethers/src/internal/ethers-provider-wrapper.ts:13:20)


In [3]:
next(async()=>{   
    tickTime = $.time(5,'days')
    uniResponse = await uniV3Pool.observe([0,tickTime])
    poolLiquidity = await uniV3Pool.liquidity()
    
    quote = 1.0001**(((uniResponse[0][1]).sub(uniResponse[0][0])).div(tickTime))
    
    console.log('previous liquidity', bnToNumber(poolLiquidity))
    console.log('previous tickQuote', 1/quote)
})

TransactionExecutionError: Transaction ran out of gas
    at HardhatNode._manageErrors (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/internal/hardhat-network/provider/node.js:870:20)
    at HardhatNode.estimateGas (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/internal/hardhat-network/provider/node.js:297:35)
    at async EthModule._estimateGasAction (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/internal/hardhat-network/provider/modules/eth.js:225:67)
    at async HardhatNetworkProvider.request (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/internal/hardhat-network/provider/provider.js:71:26)
    at async EthersProviderWrapper.send (/Users/levihernan/wonder/keep3r/notebooks/node_modules/@nomiclabs/hardhat-ethers/dist/src/internal/ethers-provider-wrapper.js:11:24)
TransactionExecutionError: Transaction ran out of gas
    at HardhatNode._manageErrors (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/inter

In [4]:
next(async()=>{
    // mints 100 KP3R and 100 WETH to provider and approves spendings
    await keep3r.proxy.connect(keep3r.governance)['mint(address,uint256)'](provider.address, oneHundred)
    await keep3r.v1.connect(provider).approve(keep3r.pool.address, oneHundred)
    await weth.connect(provider).deposit(oneHundred, {value:oneHundred})
    await weth.connect(provider).approve(keep3r.pool.address, oneHundred)
    
    kp3rInitialBalance = await keep3r.v1.balanceOf(provider.address)
    wethInitialBalance = await weth.balanceOf(provider.address)

    // MINT
    liquidity = await keep3r.pool.connect(provider).callStatic.mint(oneHundred,oneHundred,0,0,provider.address)
    await keep3r.pool.connect(provider).mint(oneHundred,oneHundred,0,0,provider.address)
    
    kp3rBalance = await keep3r.v1.balanceOf(provider.address)
    wethBalance = await weth.balanceOf(provider.address)
    klpBalance = await keep3r.pool.balanceOf(provider.address)
    console.log('KP3R spent', bnToNumber(kp3rInitialBalance.sub(kp3rBalance)))
    console.log('WETH spent', bnToNumber(wethInitialBalance.sub(wethBalance)))
    console.log('kLP minted', bnToNumber(klpBalance))
})

TypeError: Cannot read property 'connect' of undefined
    at evalmachine.<anonymous>:3:24
    at next (/Users/levihernan/wonder/keep3r/notebooks/utils/jupyter.js:3:53)
    at evalmachine.<anonymous>:1:1
    at Script.runInThisContext (node:vm:129:12)
    at Object.runInThisContext (node:vm:305:38)
    at run ([eval]:1054:15)
    at onRunRequest ([eval]:888:18)
    at onMessage ([eval]:848:13)
    at process.emit (node:events:394:28)
    at emit (node:internal/child_process:917:12)


In [44]:
quote

18.346788208977543

In [48]:
(oneHundred / (quote)**0.5)/10**18

23.34640290661574

In [51]:
0x01440295848ef91d29/10**18

23.347387814808986

In [39]:
100/(18)**(0.5)

23.570226039551585

In [8]:
keep3r.pool.slot0()

TypeError: keep3r.pool.slot0 is not a function

In [6]:
keep3r.pool.callStatic.viewPosition()

'0xd47863c2d017c4628bf92851402add49493602bb37103452aae69ce6b7a1563a'

In [10]:
uniV3Pool.slot0()

[
  BigNumber { _hex: '0x3bc4f1af1dc82080f71fa9f5', _isBigNumber: true },
  -29096,
  0,
  1,
  1,
  0,
  true,
  sqrtPriceX96: BigNumber { _hex: '0x3bc4f1af1dc82080f71fa9f5', _isBigNumber: true },
  tick: -29096,
  observationIndex: 0,
  observationCardinality: 1,
  observationCardinalityNext: 1,
  feeProtocol: 0,
  unlocked: true
]

In [7]:
keep3r.pool.principal(10)

Error: Transaction reverted without a reason string

In [15]:
0x04

4

In [6]:
uniV3Pool.positions('0xd47863c2d017c4628bf92851402add49493602bb37103452aae69ce6b7a1563a')

[
  BigNumber { _hex: '0x01440295848ef91d29', _isBigNumber: true },
  BigNumber { _hex: '0x00', _isBigNumber: true },
  BigNumber { _hex: '0x00', _isBigNumber: true },
  BigNumber { _hex: '0x00', _isBigNumber: true },
  BigNumber { _hex: '0x00', _isBigNumber: true },
  _liquidity: BigNumber { _hex: '0x01440295848ef91d29', _isBigNumber: true },
  feeGrowthInside0LastX128: BigNumber { _hex: '0x00', _isBigNumber: true },
  feeGrowthInside1LastX128: BigNumber { _hex: '0x00', _isBigNumber: true },
  tokensOwed0: BigNumber { _hex: '0x00', _isBigNumber: true },
  tokensOwed1: BigNumber { _hex: '0x00', _isBigNumber: true }
]

In [6]:
next(async()=>{
    /* Notice:
        at current block, pools cardinality is 1
        provider have just minted a position in the pool
        latest twap accumulator is 1s ago
        UniswapV3Pool will return 'OLD' and provide no credits
    */ 
    await advanceTimeAndBlock(432000)
})

In [8]:
// Make a small swap
next(async()=>{
    uniRouter = await ethers.getContractAt('ISwapRouter','0xE592427A0AEce92De3Edee1F18E0157C05861564')
    
    await keep3r.proxy.connect(keep3r.governance)['mint(address,uint256)'](provider.address, toUnit(100))
    await keep3r.v1.connect(provider).approve(uniRouter.address,toUnit(100))
    
    await uniRouter.connect(provider).exactInputSingle([
                keep3r.v1.address,
                '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
                10000,
                keep3r.v1.address,
                1628713433+100000000,
                toUnit(100),
                toUnit(0.0001),
                0])
})

In [21]:
uniV3Pool.liquidity()

BigNumber { _hex: '0x01440295848ef91d29', _isBigNumber: true }

In [22]:
0x01440295848ef91d29/10**18

23.347387814808986

In [9]:
next(async()=>{    
//     await keep3r.v2.connect(provider).addJob(job.address)
    await keep3r.pool.connect(provider).approve(keep3r.v2.address, klpBalance)
    await keep3r.v2.connect(provider).addLiquidityToJob(job.address, keep3r.pool.address, klpBalance)
})

In [10]:
next(async()=>{
    console.log('quoteLiquidity', (await keep3r.v2.quoteLiquidity(keep3r.pool.address,klpBalance)).toString())
    console.log('jobPeriodCredits', (await keep3r.v2.jobPeriodCredits(job.address)).toString())
    console.log('jobLiquidityCredits', (await keep3r.v2.jobLiquidityCredits(job.address)).toString())
})

quoteLiquidity 62992585216367744469
jobPeriodCredits 62992585216367744469
jobLiquidityCredits 0


In [11]:
62992585216367744469/10**18

62.99258521636775

In [12]:
100 * 5 / 34

14.705882352941176